In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
class BBoxSimilaritySearch(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, exemplar, search_target):
        exe_embed = self.embedder(exemplar)
        target_embed = self.embedder(search_target)
        ans = self.searcher(exe_embed, target_embed)
        # hmm, maybe one backbone and multiple heads?
        # would Centernet itself work well here?
        # concat CenterNet features with Siamese?
        # recurrent bbox controller?
        #  adaptive computation time?
        # higher order training?
        
        return x1, y1, x2, y2